# Collections of functions to be used as init script

In [4]:
import sys
import pandas as pd
import numpy as np
import numpy.ma as ma
import time
import matplotlib
from colorsys import hsv_to_rgb
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator, FormatStrFormatter, MaxNLocator
import matplotlib.patches as patches
import matplotlib.dates as md
import matplotlib.dates as mdates
import datetime
import sympy as sp
import seaborn as sns
import itertools
import pickle
import gzip
import os
from scipy.interpolate import interp1d
from collections import OrderedDict
import glob
from pprint import pprint
import pytz
from scipy.signal import savgol_filter
from operator import itemgetter
import matplotlib.gridspec as gridspec

import cl2pd
from cl2pd import importData
from cl2pd import plotFunctions
from cl2pd import dotdict
from cl2pd import bbFunctions
from cl2pd import variablesDF
cals=importData.cals

import warnings
warnings.filterwarnings("ignore")
print('/eos/user/n/nkarast/myLibrary/init.ipynb :: Ignoring warnings.')

/eos/user/n/nkarast/myLibrary/init.ipynb :: Ignoring warnings.


In [1]:
%run -i ~/myLibrary/resonanceLines.ipynb

ERROR:root:File `u'/eos/user/n/nkarast/myLibrary/resonanceLines.ipynb.py'` not found.


# Some Global Settings

In [4]:
_timezone = pytz.timezone("Europe/Zurich")

sns.set_style("white")
sns.set_style("ticks")
sns.set_context("paper")

plt.rcParams['axes.labelsize']   = 18
plt.rcParams['axes.labelweight'] = 'normal'
plt.rcParams['xtick.labelsize']  = 16
plt.rcParams['ytick.labelsize']  = 16
plt.rcParams['legend.fontsize']  = 16
plt.rcParams['figure.titlesize'] = 20

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

get_ipython().magic('matplotlib inline')

# And the functions begin...

In [5]:
def setJupyterWidth():
    '''Sets Jupyter Notebook width to 100% via HTML'''
    from IPython.core.display import display, HTML
    display(HTML("<style>.container { width:100% !important; }</style>"))

In [6]:
def convertNumpyToTimestamp(x):
    '''converts numpy to timestamp'''
    try:
        return pd.DatetimeIndex(x)[0]
    except:
        return x

In [7]:
def getFilledSlotsArray(dict_intervals_two_beams, beam, cycle, cycleTime,  mask_invalid=True):
    '''
    Returns the array with the filled slots.
    Inputs : beam           : beam string ('beam_1', 'beam_2')
             cycle          : cycle string ('injection', 'flattop')
             cycleTime      : cycle step string ('injection_start', 'injection_end', 'flattop_start', 'flattop_end')
             mask_invalid   : boolean True/False to mask invalid values in the output array
    Returns: filled_slots array
    '''
    return ma.masked_invalid(dict_intervals_two_beams[beam][cycle]['filled_slots'])

In [8]:
def convertToLocalTime(df_timeRange, timezone="Europe/Zurich", scalar=False):
    '''
    Converts a Series (Pandas Column) in Local Time and returns it.
    Input  : df_timeRange : Pandas Series with timestamp data
             timezone     : timezone (tz) string - defaults to Europe/Zurich
             scalar 	  : if it is a single value and not list/array
    Returns: converted pandas Series
    '''
    if isinstance(df_timeRange, list) or isinstance(df_timeRange, np.ndarray) or isinstance(df_timeRange, pd.core.series.Series):
        return np.array(pd.to_datetime(np.array(df_timeRange), unit='s', utc=True).tz_convert(timezone).tz_localize(None))
    else:
        return datetime.datetime.utcfromtimestamp(df_timeRange) + (datetime.datetime.now()-datetime.datetime.utcnow()) # + utc offset to fix local zone



In [9]:
def plot_mean_and_spread(axc, x_vect, ymat, color = 'k', alpha=1, label=None, shade=False):
    '''
    Function to plot the average value and the +/- 2sigma range of a given
    vector on a given axis

    Inputs : axc : axis on which you want me to plot stuff
             x_vect: array of x-axis values (any iterable object)
             y_mat : array of y-axis values for which the std and mean will be Calculated
             color : the color for the lines
             alpha : the alpha of the lines
             label : label to add to this axis
             shade : plot in shade the +/- 1 standard deviation band
    Returns: None -- simply adds stuff on the given pl.axis
    '''
    avg = np.nanmean(ymat, axis=1)
    std = np.nanstd(ymat, axis=1)

    if shade:
        axc.fill_between(x_vect, avg-std, avg+std, alpha=.3, color=color, label=None)
    else:
        axc.plot(x_vect, avg-1*std, '--', color=color, linewidth=1, alpha=alpha, label=None)
        axc.plot(x_vect, avg+1*std, '--', color=color, linewidth=1, alpha=alpha, label=None)
        axc.plot(x_vect, avg, color=color, linewidth=2, alpha=alpha, label=label)

In [10]:
def colorprog(i_prog, Nplots, v1 = .9, v2 = 1.):
    '''
    Function to return different colors for bunch plots
    '''
    if hasattr(Nplots, '__len__'):
        Nplots = len(Nplots)
    return hsv_to_rgb(float(i_prog)/float(Nplots), v1, v2)


In [ ]:
def getLHCSuperTable(fill_min=None, fill_max=None, t1=None, t2=None):
    '''
    Returns a "SuperTable" for fills within fill_min, fill_max or in time range t1,t2
    '''
    st_vars = ['LHC.STATS:FILL_NUMBER', 'LHC.STATS:ENERGY', 'LHC.STATS:LHC:INJECTION_SCHEME', 'LHC.STATS:B1_NUMBER_BUNCHES', 'LHC.STATS:NUMBER_COLLISIONS_IP1_5', 'LHC.STATS:NUMBER_COLLISIONS_IP2', 'LHC.STATS:NUMBER_COLLISIONS_IP8',
               'LHC.STATS:BETA_STAR_ALICE', 'LHC.STATS:BETA_STAR_ATLAS', 'LHC.STATS:BETA_STAR_CMS', 'LHC.STATS:BETA_STAR_LHCB',  'LHC.STATS:LUMI_ATLAS_DELIVERED', 'LHC.STATS:LUMI_ATLAS_PEAK', 'LHC.STATS:LUMI_CMS_DELIVERED', 'LHC.STATS:LUMI_CMS_PEAK',
               'LHC.STATS:LHCB_HALF_CROSSING_ANGLE', 'LHC.STATS:ALICE_HALF_CROSSING_ANGLE', 'LHC.STATS:ATLAS_HALF_CROSSING_ANGLE', 'LHC.STATS:CMS_HALF_CROSSING_ANGLE', 
               'LHC.STATS:SPS_H_ROT_IN:EMITTANCE', 'LHC.STATS:SPS_V_ROT_IN:EMITTANCE']
    
    if fill_min is not None and fill_max is not None and t1 is None and t2 is None:
        print "Running SuperTable by fills [{}, {}]...".format(fill_min, fill_max)
        log = pytimber.LoggingDB()
        d1 = log.getLHCFillData(fill_min, True)
        t1 = convertToLocalTime(d1['startTime'])
        d2 = log.getLHCFillData(fill_max, True)
        t2 = convertToLocalTime(d2['endTime'])
        print "Corresponding start/end time is : [{}, {}]".format(t1, t2)
    elif fill_min is None and fill_max is None and t1 is not None and t2 is not None:
        print "Running SuperTable by time [{},{}]...".format(t1, t2)
    else:
        print "Getting SuperTable for the last five days..."
        t1 = datetime.datetime.now() - datetime.timedelta(days=5)
        t2 = datetime.datetime.now()
        
    df = myToolbox.cals2pnd(st_vars, t1,t2)
    df.columns = [x.split('LHC.STATS:')[1] for x in df.columns.tolist()]
    df = df.fillna(method='ffill').fillna(method='bfill')
    return df.set_index(['FILL_NUMBER'])
    

In [11]:
def convertNumpyDatetimeToDatetime(times):
    '''
    converts the np.datetime64 to datetime.datetime at local time
    returns an array object
    '''
    result = np.array([x.replace(tzinfo=pytz.utc).astimezone(_timezone) for x in map(datetime.datetime.utcfromtimestamp, times.astype(np.int64)/1e9)])
    if len(result)==1:
        return result[0]
    else:
        return result

In [12]:
class pp():
    "This an auxillary class to allow for LaTeX output between subsequent derivation steps."
    def __init__(self, eq, left=None):
        self.eq = eq
        self.left = left
    def _repr_html_(self):
        if self.left:
            return "$" + sp.latex(self.left) + " = " + sp.latex(self.eq) + "$"
        return "$" + sp.latex(self.eq) + "$"

In [14]:
def printBSRTCalibrationFactor(filln):
    '''given a fill prints out the calibration factors (mean over FB/FT)'''
    tf1, tf2 = getLHCFillTimes(filln)
    fills, bmodes = myToolbox.LHCFillsByTime2pnd(tf1, tf2)
    
    t_start_inj   = bmodes['startTime'][bmodes['mode']=='INJPHYS'].values[0]
    t_start_ramp  = bmodes['endTime'][bmodes['mode']=='PRERAMP'].values[0]
    t_start_ft    = bmodes['startTime'][bmodes['mode']=='FLATTOP'].values[0]
    t_end_sb      = bmodes['endTime'][bmodes['mode']=='STABLE'].values[0]
    
    dt_start_inj  = datetime.datetime.utcfromtimestamp(t_start_inj.tolist()/1e9)
    dt_start_ramp = datetime.datetime.utcfromtimestamp(t_start_ramp.tolist()/1e9)
    dt_start_ft   = datetime.datetime.utcfromtimestamp(t_start_ft.tolist()/1e9)
    dt_end_sb     = datetime.datetime.utcfromtimestamp(t_end_sb.tolist()/1e9)
    
    beam_vars = { 1:['LHC.BSRT.5R4.B1:BETA_H', 
                      'LHC.BSRT.5R4.B1:BETA_V', 
                      'LHC.BSRT.5R4.B1:SCALE_H', 
                      'LHC.BSRT.5R4.B1:SCALE_V', 
                      'LHC.BSRT.5R4.B1:LSF_H', 
                      'LHC.BSRT.5R4.B1:LSF_V', 
              ],

             2: [ 'LHC.BSRT.5L4.B2:BETA_H', 
                  'LHC.BSRT.5L4.B2:BETA_V', 
                  'LHC.BSRT.5L4.B2:SCALE_H', 
                  'LHC.BSRT.5L4.B2:SCALE_V', 
                  'LHC.BSRT.5L4.B2:LSF_H', 
                  'LHC.BSRT.5L4.B2:LSF_V', 
              ],
            }
    dfs = {1: {'FB': None, 'FT': None},
           2: {'FB': None, 'FT': None}}
    for beam_n in [1,2]:
        for tag in ['FB', 'FT']:
            print("Working on beam-{} {}".format(beam_n, tag))
            tmp_vars = beam_vars[beam_n]
            tmp_df = pd.DataFrame()

            if tag == 'FB':
                t1 = dt_start_inj
                t2 = dt_start_ramp
            elif tag == 'FT':
                t1 = dt_start_ft
                t2 = dt_end_sb

            tmp_df = myToolbox.cals2pnd(tmp_vars, t1, t2)
            dfs[beam_n][tag]=tmp_df
    print('''
            for kk in e_dict.keys():
                e_dict[kk] = {450:{}, 6500:{}}'''+'''
            
            e_dict['betaf_h'][450][1]       = {}
            e_dict['betaf_h'][6500][1]      = {}
                
            e_dict['betaf_v'][450][1]       = {}
            e_dict['betaf_v'][6500][1]      = {}
                
            e_dict['sigma_corr_h'][450][1]  = {}
            e_dict['sigma_corr_h'][6500][1] = {}
                
            e_dict['sigma_corr_v'][450][1]  = {}
            e_dict['sigma_corr_v'][6500][1] = {}
                
            e_dict['scale_h'][450][1]       = {}
            e_dict['scale_h'][6500][1]      = {}
                
            e_dict['scale_v'][450][1]       = {}
            e_dict['scale_v'][6500][1]      = {}
                
            e_dict['rescale_sigma_h'][450][1]  = 1.
            e_dict['rescale_sigma_h'][6500][1] = 1.
            e_dict['rescale_sigma_v'][450][1]  = 1.
            e_dict['rescale_sigma_v'][6500][1] = 1.
                
            # Beam 2:
            e_dict['betaf_h'][450][2]       = {}
            e_dict['betaf_h'][6500][2]      = {}
                
            e_dict['betaf_v'][450][2]       = {}
            e_dict['betaf_v'][6500][2]      = {}
                
            e_dict['sigma_corr_h'][450][2]  = {}
            e_dict['sigma_corr_h'][6500][2] = {}
                
            e_dict['sigma_corr_v'][450][2]  = {}
            e_dict['sigma_corr_v'][6500][2] = {}
                
            e_dict['scale_h'][450][2]       = {}
            e_dict['scale_h'][6500][2]      = {}
                
            e_dict['scale_v'][450][2]       = {}
            e_dict['scale_v'][6500][2]      = {}
                
            e_dict['rescale_sigma_h'][450][2]  = 1.
            e_dict['rescale_sigma_h'][6500][2] = 1.
            e_dict['rescale_sigma_v'][450][2]  = 1.
            e_dict['rescale_sigma_v'][6500][2] = 1.
            
            # gamma
            e_dict['gamma'][450] = 479.6
            e_dict['gamma'][6500] = 6927.6

            print('Using calibration TEST')
                '''.format(dfs[1]['FB']['LHC.BSRT.5R4.B1:BETA_H'].mean(), 
                           dfs[1]['FT']['LHC.BSRT.5R4.B1:BETA_H'].mean(), 
                           dfs[1]['FB']['LHC.BSRT.5R4.B1:BETA_V'].mean(), 
                           dfs[1]['FT']['LHC.BSRT.5R4.B1:BETA_V'].mean(), 
                           dfs[1]['FB']['LHC.BSRT.5R4.B1:LSF_H'].mean(), 
                           dfs[1]['FT']['LHC.BSRT.5R4.B1:LSF_H'].mean(), 
                           dfs[1]['FB']['LHC.BSRT.5R4.B1:LSF_V'].mean(), 
                           dfs[1]['FT']['LHC.BSRT.5R4.B1:LSF_V'].mean(), 
                           dfs[1]['FB']['LHC.BSRT.5R4.B1:SCALE_H'].mean(), 
                           dfs[1]['FT']['LHC.BSRT.5R4.B1:SCALE_H'].mean(), 
                           dfs[1]['FB']['LHC.BSRT.5R4.B1:SCALE_V'].mean(), 
                           dfs[1]['FT']['LHC.BSRT.5R4.B1:SCALE_V'].mean(), 
                           dfs[2]['FB']['LHC.BSRT.5L4.B2:BETA_H'].mean(), 
                           dfs[2]['FT']['LHC.BSRT.5L4.B2:BETA_H'].mean(), 
                           dfs[2]['FB']['LHC.BSRT.5L4.B2:BETA_V'].mean(), 
                           dfs[2]['FT']['LHC.BSRT.5L4.B2:BETA_V'].mean(), 
                           dfs[2]['FB']['LHC.BSRT.5L4.B2:LSF_H'].mean(), 
                           dfs[2]['FT']['LHC.BSRT.5L4.B2:LSF_H'].mean(), 
                           dfs[2]['FB']['LHC.BSRT.5L4.B2:LSF_V'].mean(), 
                           dfs[2]['FT']['LHC.BSRT.5L4.B2:LSF_V'].mean(), 
                           dfs[2]['FB']['LHC.BSRT.5L4.B2:SCALE_H'].mean(), 
                           dfs[2]['FT']['LHC.BSRT.5L4.B2:SCALE_H'].mean(), 
                           dfs[2]['FB']['LHC.BSRT.5L4.B2:SCALE_V'].mean(), 
                           dfs[2]['FT']['LHC.BSRT.5L4.B2:SCALE_V'].mean()))    

In [2]:
def getFillingScheme(filln):
    '''Given a fill number return:
    filled_slots_b1, filled_slots_b2, slots_coll_b1, slots_coll_b2, slots_noncoll_b1, slots_noncoll_b2, b1_fill_pattern, b2_fill_pattern
    '''
    
    bmodes = importData.LHCFillsByNumber(filln)
    FILL_PATTERN_DF = pd.DataFrame()
    FILL_PATTERN_DF = importData.cals2pd(['LHC.BCTFR.A6R4.B1:BUNCH_FILL_PATTERN','LHC.BCTFR.A6R4.B2:BUNCH_FILL_PATTERN'], pd.Timestamp(bmodes['startTime'][bmodes['mode']=='PRERAMP'].values[0]), pd.Timestamp(bmodes['startTime'][bmodes['mode']=='PRERAMP'].values[0]+pd.Timedelta(seconds=1)) )
    b1_fill_pattern = np.array(FILL_PATTERN_DF.filter(regex="LHC.*B1.*").iloc[0].values[0]);
    b2_fill_pattern = np.array(FILL_PATTERN_DF.filter(regex="LHC.*B2.*").iloc[0].values[0]);

    # define the filled slots and the colliding / non-colliding
    filled_slots_b1 = np.where(b1_fill_pattern == 1)[0]
    filled_slots_b2 = np.where(b2_fill_pattern == 1)[0]
    slots_noncoll_b1 = np.array([x for x in filled_slots_b1 if x not in filled_slots_b2])
    slots_noncoll_b2 = np.array([x for x in filled_slots_b2 if x not in filled_slots_b1])
    slots_coll_b1 = np.array([x for x in filled_slots_b1 if x in filled_slots_b2])
    slots_coll_b2 = np.array([x for x in filled_slots_b2 if x in filled_slots_b1])

    print('---------- FILL {} ----------'.format(filln))
    print('B1 slots               : {}'.format(len(filled_slots_b1)))
    print('B2 slots               : {}'.format(len(filled_slots_b2)))
    print('B1 colliding slots     : {}'.format(len(slots_coll_b1)))
    print('B2 colliding slots     : {}'.format(len(slots_coll_b2)))
    print('B1 non-colliding slots : {}'.format(len(slots_noncoll_b1)))
    print('B2 non-colliding slots : {}'.format(len(slots_noncoll_b2)))
    return filled_slots_b1, filled_slots_b2, slots_coll_b1, slots_coll_b2, slots_noncoll_b1, slots_noncoll_b2, b1_fill_pattern, b2_fill_pattern
        

In [16]:
def plotFillingScheme(filln):
    filled_slots_b1, filled_slots_b2, slots_coll_b1, slots_coll_b2, slots_noncoll_b1, slots_noncoll_b2, b1_fill_pattern, b2_fill_pattern = getFillingScheme(filln)
    fig1 = plt.figure('fillPattern', figsize=(12,9))
    ax1 = plt.subplot(211)       
    ax1.step(np.arange(len(b1_fill_pattern)), b1_fill_pattern, where='post', color='b', lw=2);
    plt.yticks([0,1], ['EMPTY', 'FILLED']);
    ax1.set_xlim(-100, 3570);
    ax1.set_ylim(0, 1.1);
    ax1.set_ylabel('BEAM 1', fontsize=18);
    ax1.set_title('FILL {}'.format(filln), fontsize=20);
    
    
    # -- beam 2
    ax2 = plt.subplot(212, sharex=ax1, sharey=ax1);
    ax2.step(np.arange(len(b2_fill_pattern)), b2_fill_pattern, where='post', color='r', lw=2);
    ax2.set_ylabel('BEAM 2', fontsize=18);
    plt.yticks([0,1], ['EMPTY', 'FILLED']);
    ax2.set_xlabel('Bunch Slot [25ns]', fontsize=16);

In [17]:
def getBeamBatches(filln):
    '''Given a fill number, aquire the filling pattern and return a list of lists for b1_trains, b2_trains'''
    filled_slots_b1, filled_slots_b2, slots_coll_b1, slots_coll_b2, slots_noncoll_b1, slots_noncoll_b2, b1_fill_pattern, b2_fill_pattern = getFillingScheme(filln)
    b1_trains, b2_trains = [], []
    for k,g in itertools.groupby(enumerate(filled_slots_b1),lambda x:x[0]-x[1]):
        group = (map(itemgetter(1),g))
        group = list(map(int,group))
        b1_trains.append(group)
    for k,g in itertools.groupby(enumerate(filled_slots_b2),lambda x:x[0]-x[1]):
        group = (map(itemgetter(1),g))
        group = list(map(int,group))
        b2_trains.append(group)#ranges.append((group[0],group[-1]))
        
    return b1_trains, b2_trains
    

In [8]:
def getNewColor(nmax):
    '''given: nmax, n (maximum length and current id) return a new color to be used in loop'''
    color=iter(plt.cm.rainbow(np.linspace(0,1,nmax)))
    colors = []
    for i in xrange(nmax):
        c=next(color)
        colors.append(c)
        
    return colors

In [ ]:
def formatDatetimeAxis(ax, which='x', fmt="%H:%M"):
    '''Sets the format datetime axis of (ax) of (which=x|y|both) axis to fmt ("%H:%M")
    '''
    if which=='x':
        ax.xaxis.set_major_formatter(mdates.DateFormatter(fmt))
    elif which == 'y':
        ax.yaxis.set_major_formatter(mdates.DateFormatter(fmt))
    elif which =='both':
        ax.xaxis.set_major_formatter(mdates.DateFormatter(fmt))
        ax.yaxis.set_major_formatter(mdates.DateFormatter(fmt))
    else:
        return

In [ ]:
def setAxisMaxLocator(ax, which='y', nloc=10):
    '''Sets the maxlocator of (ax) of (which=x|y|both) axis to nloc (10)
    '''
    if which=='y':
        ax.yaxis.set_major_locator(MaxNLocator(nloc))
    elif which=='x':
        ax.xaxis.set_major_locator(MaxNLocator(nloc))
    elif which=='both':
        ax.xaxis.set_major_locator(MaxNLocator(nloc))
        ax.yaxis.set_major_locator(MaxNLocator(nloc))
    else:
        return

In [3]:
def getEncounterSchedule(filln, nInsideD1=5, nLR=[16, 16, 15]):
    '''
    Returns the encounter pattern:
    ndf_b1, ndf_b2 = collision schedules for b1, b2
    '''
    filled_slots_b1, filled_slots_b2, slots_coll_b1, slots_coll_b2, slots_noncoll_b1, slots_noncoll_b2, b1_fill_pattern, b2_fill_pattern = getFillingScheme(filln)
    BBMATRIX = bbFunctions.computeBBMatrix(nLR)
    b1_collisionScheduleDF = bbFunctions.B1CollisionScheduleDF(b1_fill_pattern, b2_fill_pattern, nLR)
    b2_collisionScheduleDF = bbFunctions.B2CollisionScheduleDF(b1_fill_pattern, b2_fill_pattern, nLR)
    bbencounters = bbFunctions.BBEncounterSchedule(filled_slots_b1, filled_slots_b2, BBMATRIX)
    

    # Collides in IP2
    # -- B1
    b1_collisionScheduleDF['collidesIP2'] = b1_collisionScheduleDF["HO partner in ALICE"].fillna(-1)
    b1_collisionScheduleDF['collidesIP2'][b1_collisionScheduleDF['collidesIP2']>=0] = 2
    # -- B2
    b2_collisionScheduleDF['collidesIP2'] = b2_collisionScheduleDF["HO partner in ALICE"].fillna(-1)
    b2_collisionScheduleDF['collidesIP2'][b2_collisionScheduleDF['collidesIP2']>=0] = 2

    # Collides in IP8
    # -- B1
    b1_collisionScheduleDF['collidesIP8'] = b1_collisionScheduleDF["HO partner in LHCB"].fillna(-1)
    b1_collisionScheduleDF['collidesIP8'][b1_collisionScheduleDF['collidesIP8']>=0] = 3
    # -- B2
    b2_collisionScheduleDF['collidesIP8'] = b2_collisionScheduleDF["HO partner in LHCB"].fillna(-1)
    b2_collisionScheduleDF['collidesIP8'][b2_collisionScheduleDF['collidesIP8']>=0] = 3

    # Collides in IP1/5
    # -- B1
    b1_collisionScheduleDF['collidesIP1'] = b1_collisionScheduleDF["HO partner in ATLAS/CMS"].fillna(-1)
    b1_collisionScheduleDF['collidesIP1'][b1_collisionScheduleDF['collidesIP1']>=0] = 1
    # -- B2
    b2_collisionScheduleDF['collidesIP1'] = b2_collisionScheduleDF["HO partner in ATLAS/CMS"].fillna(-1)
    b2_collisionScheduleDF['collidesIP1'][b2_collisionScheduleDF['collidesIP1']>=0] = 1
    
    ndf_b1 = pd.DataFrame()
    ndf_b2 = pd.DataFrame()
    
    ndf_b1['IP15'] = b1_collisionScheduleDF['# of LR in ATLAS/CMS']
    ndf_b1['IP2']  = b1_collisionScheduleDF['# of LR in ALICE']
    ndf_b1['IP8']  = b1_collisionScheduleDF['# of LR in LHCB']
    ndf_b1.index   = b1_collisionScheduleDF.index

    ndf_b2['IP15'] = b2_collisionScheduleDF['# of LR in ATLAS/CMS']
    ndf_b2['IP2']  = b2_collisionScheduleDF['# of LR in ALICE']
    ndf_b2['IP8']  = b2_collisionScheduleDF['# of LR in LHCB']
    ndf_b2.index   = b2_collisionScheduleDF.index
    
    return ndf_b1, ndf_b2